In [1]:
import pandas as pd
import numpy as np
import langgraph

In [5]:
data = pd.read_csv("data_science_interview_questions.csv")

In [7]:
data.difficulty.value_counts()

difficulty
Hard      71
Medium    65
Easy      64
Name: count, dtype: int64

In [9]:
data.category.value_counts()

category
Deep Learning       71
Machine Learning    69
Statistics          60
Name: count, dtype: int64

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, List, Literal
import random

# Question types
Difficulty = Literal["Easy", "Medium", "Hard"]
Category = Literal["Machine Learning", "Deep Learning", "Statistics"]

# LangGraph state
class TeachingState(TypedDict):
    question: str
    category: Category
    difficulty: Difficulty
    student_reply: str
    teacher_reply: str
    dean_feedback: str
    revised_teacher_reply: str
    cognitive_state: dict

# Simulated student agent
def student_agent(state: TeachingState) -> TeachingState:
    simulated_answers = {
        "Easy": "I think it's about averages.",
        "Medium": "Not sure, maybe it's like combining trees?",
        "Hard": "Hmm... does this involve eigenvalues?"
    }
    state["student_reply"] = simulated_answers[state["difficulty"]]
    return state

# Simulated teacher agent (provoking style)
def teacher_agent(state: TeachingState) -> TeachingState:
    socratic_responses = {
        "Easy": "Interesting, but what happens if you consider more examples?",
        "Medium": "You're close—can you relate this to overfitting?",
        "Hard": "Good try—what's the role of assumptions in this context?"
    }
    state["teacher_reply"] = socratic_responses[state["difficulty"]]
    return state

# Dean agent (judges and revises)
def dean_agent(state: TeachingState) -> TeachingState:
    feedback = "Too direct" if "consider" not in state["teacher_reply"] else "Acceptable"
    state["dean_feedback"] = feedback

    if feedback == "Too direct":
        state["revised_teacher_reply"] = "Instead of answering, reflect: what conditions must be true here?"
    else:
        state["revised_teacher_reply"] = state["teacher_reply"]

    return state

# Cognitive state tracker
def cognitive_state_system(state: TeachingState) -> TeachingState:
    dimensions = ["Problem Understanding", "Instruction Understanding", "Calculation", "Knowledge Mastery", "Thirst for Learning"]
    state["cognitive_state"] = {dim: random.choice(["Low", "Medium", "High"]) for dim in dimensions}
    return state


In [3]:
builder = StateGraph(TeachingState)

builder.add_node("Student", student_agent)
builder.add_node("Teacher", teacher_agent)
builder.add_node("Dean", dean_agent)
builder.add_node("CognitiveState", cognitive_state_system)

# Define edges
builder.set_entry_point("Student")
builder.add_edge("Student", "Teacher")
builder.add_edge("Teacher", "Dean")
builder.add_edge("Dean", "CognitiveState")
builder.add_edge("CognitiveState", END)

graph = builder.compile()

In [4]:
sample_state = {
    "question": "What is overfitting in machine learning?",
    "category": "Machine Learning",
    "difficulty": "Easy",
    "student_reply": "",
    "teacher_reply": "",
    "dean_feedback": "",
    "revised_teacher_reply": "",
    "cognitive_state": {}
}

output = graph.invoke(sample_state)
print(output)


{'question': 'What is overfitting in machine learning?', 'category': 'Machine Learning', 'difficulty': 'Easy', 'student_reply': "I think it's about averages.", 'teacher_reply': 'Interesting, but what happens if you consider more examples?', 'dean_feedback': 'Acceptable', 'revised_teacher_reply': 'Interesting, but what happens if you consider more examples?', 'cognitive_state': {'Problem Understanding': 'Low', 'Instruction Understanding': 'Low', 'Calculation': 'Low', 'Knowledge Mastery': 'Low', 'Thirst for Learning': 'Low'}}


In [5]:
def generate_student_reply(question: str, persona: str, model) -> str:
    prompt = f"""
You are a data science student with the following traits:
- Persona: {persona}
- Problem Understanding: {persona_traits[persona]["Problem Understanding"]}
- Instruction Understanding: {persona_traits[persona]["Instruction Understanding"]}
- Calculation: {persona_traits[persona]["Calculation"]}
- Knowledge Mastery: {persona_traits[persona]["Knowledge Mastery"]}
- Thirst for Learning: {persona_traits[persona]["Thirst for Learning"]}

Respond to this question with your best attempt: "{question}"
"""
    return model(prompt)  # Replace with actual model call


In [6]:
def generate_teacher_reply(question: str, student_reply: str, model) -> str:
    prompt = f"""
As a Socratic teacher, respond to the student reply with a question that encourages deeper thinking and reflection. 
Don't give the answer directly.

Question: "{question}"
Student's Attempt: "{student_reply}"

Socratic Response:
"""
    return model(prompt)


In [7]:
def evaluate_teacher_response(question: str, student_reply: str, teacher_reply: str, model) -> dict:
    prompt = f"""
Evaluate the teacher's Socratic response below. Does it follow Socratic principles?
- It should not give direct answers.
- It should guide through inquiry.
- It should point out student's misunderstanding if any.

Question: "{question}"
Student: "{student_reply}"
Teacher: "{teacher_reply}"

Return:
- "verdict": "Acceptable" or "Too direct"
- "revision": a more Socratic version if needed
"""
    return model(prompt)  # Should return a dict
